In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import pandas as pd
import numpy as np
from rdkit import Chem
from openbabel import pybel

from tqdm import tqdm
import os
from collections import defaultdict

In [3]:
import sys
sys.path.append('../')
sys.path.append('../data_processing/')
sys.path.append('../evaluation/')

In [4]:
from pharmacophore_matching import center2zero
from script_utils import load_qm9_data, load_dataset
from data_processing.ligand import Ligand
from utils_eval import extract_pp, extract_all_pp, pp_match

In [5]:
# root_path = '../lightning_logs/vp_bridge_egnn_CombinedSparseGraphDataset_2024-08-19_21_05_04.140916'
root_path = '../lightning_logs/vp_bridge_egnn_CombinedSparseGraphDataset_2024-08-19_21_05_04.140916/ligand_based'
# root_path = '../lightning_logs/vp_bridge_egnn_CombinedSparseGraphDataset_2024-08-19_13_30_50.133260'
# root_path = '../lightning_logs/vp_bridge_egnn_CombinedSparseGraphDataset_2024-08-19_13_30_50.133260/ligand_based_old'

In [6]:
aromatic = True
# we don't use optimized mols for pp matching eval
optimization = False
fn = 'reconstructed_mols' + '_aromatic_mode' if aromatic else 'reconstructed_mols'
match_fn = fn + '_matches.pkl'
score_fn = fn + '_scores.csv'
match_file = os.path.join(root_path, match_fn)
score_file = os.path.join(root_path, score_fn)
match_file

'../lightning_logs/vp_bridge_egnn_CombinedSparseGraphDataset_2024-08-19_21_05_04.140916/ligand_based/reconstructed_mols_aromatic_mode_matches.pkl'

In [7]:
gen_path = os.path.join(root_path, 'reconstructed_mols')
gen_path = gen_path + '_aromatic_mode' if aromatic else gen_path
gen_path = gen_path + '_optimized' if optimization else gen_path

In [8]:
with open(match_file, 'rb') as f:
    match_dict = pickle.load(f)

In [11]:
if 'ligand_based' in root_path:
    pp_info_file = '../../data/cleaned_crossdocked_data/metadata_HDBSCAN_non_filtered/test_pp_info.pkl'
else:
    pp_info_file = '../../data/cleaned_crossdocked_data/metadata/test_pp_info.pkl'
# pp_info_file = '../../data/cleaned_crossdocked_data/metadata/test_pp_info.pkl'
with open(pp_info_file, 'rb') as f:
    pp_info = pickle.load(f)

# pp_info

In [21]:
pp_info['4yur_A_rec_3cok_anp_lig_tt_min_0']

{'pp_atom_indices': [[[10, 11, 13, 14, 15, 16],
   [18, 19, 20, 21, 22],
   [3],
   [5],
   [12],
   [24],
   [26],
   [1],
   [9],
   [25]]],
 'pp_positions': [tensor([[ 0.9349,  0.1075,  4.4609],
          [ 0.8786,  0.0342, -3.3430],
          [-3.3192, -2.6045,  0.1178],
          [-2.9197, -0.9984,  2.2034],
          [ 2.7342,  2.1483,  4.9056],
          [-0.8664,  2.8646, -4.2189],
          [ 1.1312,  1.9272, -5.8616],
          [-0.1894, -1.7576, -0.0138],
          [ 1.7729,  1.2491,  2.0429],
          [-1.5090,  1.5094, -6.3701]])],
 'pp_types': [tensor([[0., 0., 1., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 0., 1., 0.],
          [0., 0., 0., 0., 0., 0., 1., 0.],
          [0., 0., 0., 0., 0., 0., 1., 0

In [12]:
test_dataset = load_dataset('CombinedSparseGraphDataset', '../../data/cleaned_crossdocked_data', split='test', aromatic=aromatic)
pp_info = center2zero(pp_info, test_dataset)

use processed_HDBSCAN_non_filtered
use processed_HDBSCAN_non_filtered
use processed_HDBSCAN_non_filtered
use processed_HDBSCAN_non_filtered
use processed_HDBSCAN_non_filtered


100%|██████████████████████████████████████████████████████████████████████| 15235/15235 [00:02<00:00, 7480.24it/s]


In [13]:
pp_info = dict(pp_info)

In [14]:
len(test_dataset)

15235

In [15]:
lig_list = [file.split('.')[0] for file in os.listdir(gen_path)]
len(lig_list)

14835

In [16]:
def check_match_aromatic(pp_types, pp_positions, ref_pp_info, lig, threshold=1.5):
    pps = pp_types
    ref_pps = np.argmax(ref_pp_info['pp_types'].numpy(), axis=-1)
    ref_pp_positions = ref_pp_info['pp_positions'].numpy()

    match = np.zeros_like(ref_pps)
    aro_match = False
    for i, ref_pp in enumerate(ref_pps):
        ref_pos = ref_pp_positions[i]
        for j, pp in enumerate(pps):
            if type(pp) == list:
                for pp_ in pp:
                    if pp_ == ref_pp:
                        pos = pp_positions[j]
                        dist = np.sqrt(np.sum((pos - ref_pos) ** 2))
                        if dist < threshold:
                            match[i] = 1
                            if pp_ == 2:
                                print(lig)
                                aro_match = True
                            continue
                            # match[i] += 1
            else:
                if pp == ref_pp:
                    pos = pp_positions[j]
                    dist = np.sqrt(np.sum((pos - ref_pos) ** 2))
                    if dist < threshold:
                        match[i] = 1
                        if pp == 2:
                            print(lig)
                            aro_match = True
                        continue
                        # match[i] += 1
    return match, aro_match

In [17]:
def compute_matching_scores(generated_path, pp_info, threshold=1.5):
    score_dict = {}
    match_dict = {}
    aro_match_l = []
    for file in tqdm(os.listdir(generated_path)):
        lig = file.split('.')[0]
        mol_path = os.path.join(generated_path, lig+'.sdf')
        rdmol = Chem.MolFromMolFile(mol_path, sanitize=True)
        pbmol = next(pybel.readfile("sdf", mol_path))
        try:
            rdmol = Chem.AddHs(rdmol)
            ligand = Ligand(pbmol, rdmol, atom_positions=None, conformer_axis=None, filtering=False, preprocess=False)
        except:
            print('ligand init failed')
            continue
        pp_atom_indices, pp_positions, pp_types, pp_index = extract_all_pp(ligand)
        
        ref_pp_info = pp_info[lig]
        if not all(k in list(ref_pp_info.keys()) for k in ['pp_types', 'pp_positions']):
            print(ref_pp_info)
            continue
        if isinstance(ref_pp_info['pp_types'], list):
            # print(ref_pp_info)
            ref_pp_info = {k:v[-1] for k, v in ref_pp_info.items()}     # To address the iterated list issue, for now we use the last element (which is appended to the list at last). TODO: fix this in the data processing script
        
        match, aro_match = check_match_aromatic(pp_types, pp_positions, ref_pp_info, lig, threshold=threshold)
        match_dict[lig] = match
        score = np.mean(match)
        score_dict[lig] = score
        if aro_match:
            aro_match_l.append(lig)

    return match_dict, score_dict, aro_match_l

In [18]:
match_dict, score_dict, aro_match_l = compute_matching_scores(gen_path, pp_info)

  1%|▊                                                                        | 156/14835 [00:00<00:46, 312.61it/s]

4yur_A_rec_3cok_anp_lig_tt_min_0


  4%|██▊                                                                      | 571/14835 [00:01<00:45, 311.14it/s]

3nbq_B_rec_3nbq_urf_lig_tt_docked_2


  4%|██▉                                                                      | 604/14835 [00:01<00:45, 316.20it/s]

...

2buu_A_rec_1eoc_4nc_lig_tt_min_0


 99%|██████████████████████████████████████████████████████████████████████▏| 14654/14835 [00:52<00:01, 102.72it/s][20:14:52] atom 23 has specified valence (4) smaller than the drawn valence 5.
[20:14:52] atom 24 has specified valence (1) smaller than the drawn valence 2.
*** Open Babel Warning  in ReadMolecule
The valence field specifies a valence 4 that is
less than the observed explicit valence 5.

*** Open Babel Warning  in ReadMolecule
The valence field specifies a valence 4 that is
less than the observed explicit valence 5.
The valence field specifies a valence 1 that is
less than the observed explicit valence 2.

100%|██████████████████████████████████████████████████████████████████████▋| 14764/14835 [00:52<00:00, 194.65it/s]

ligand init failed


100%|███████████████████████████████████████████████████████████████████████| 14835/14835 [00:52<00:00, 282.05it/s]

3nvv_C_rec_3nvw_gun_lig_tt_min_0


In [19]:
aro_match_l

['4yur_A_rec_3cok_anp_lig_tt_min_0',
 '3nbq_B_rec_3nbq_urf_lig_tt_docked_2',
 '4gi4_A_rec_1s38_maq_lig_tt_min_0',
 '3p1c_B_rec_3p1f_3pf_lig_tt_min_0',
 '1pw7_A_rec_1oty_6mp_lig_tt_min_0',
 '5swg_A_rec_5swt_71a_lig_tt_min_0',
 '4np3_A_rec_4np3_2l2_lig_tt_min_0',
 '4q4s_A_rec_1p0b_pq0_lig_tt_min_0',
 '4np3_A_rec_4np2_2l1_lig_tt_min_0',
 '5eyc_A_rec_3i5n_b2d_lig_tt_docked_0',
 '5bzf_A_rec_4np2_2l1_lig_tt_min_0',
 '4gg9_A_rec_1s38_maq_lig_tt_min_0',
 '5lpq_A_rec_1s38_maq_lig_tt_min_0',
 '3euf_A_rec_3nbq_urf_lig_tt_min_0',
 '2vce_A_rec_2vce_tc7_lig_tt_docked_0',
 '4xud_A_rec_5lsa_dnc_lig_tt_docked_2',
 '3nvw_C_rec_3nvw_gun_lig_tt_min_0',
 '1eoc_A_rec_2buz_4nc_lig_tt_docked_4',
 '4deh_A_rec_3i5n_b2d_lig_tt_docked_0',
 '1y5x_A_rec_4puk_2wu_lig_tt_min_0',
 '5egr_A_rec_1f3e_dpz_lig_tt_min_0',
 '5bzq_A_rec_5bzi_4wu_lig_tt_min_0',
 '2bur_A_rec_1eoc_4nc_lig_tt_min_0',
 '1k4h_A_rec_1p0b_pq0_lig_tt_docked_0',
 '5egr_A_rec_1s38_maq_lig_tt_docked_0',
 '4nyw_A_rec_4yk0_98_lig_tt_min_0',
 '3c2y_A_rec_1s

In [20]:
len(aro_match_l)

120

In [25]:
avg_score_lig_df = pd.read_csv(os.path.join(root_path, fn+'_avg_score_by_lig.csv'), header=0, index_col=0)
avg_score_rec_df = pd.read_csv(os.path.join(root_path, fn+'_avg_score_by_rec.csv'), header=0, index_col=0)
# score_by_lig

In [26]:
filtered_lig = avg_score_lig_df[(avg_score_lig_df['score'] >= 0.6) &
                                (avg_score_lig_df['num sample'] >= 10)]
filtered_lig

,score,num sample
mb3,1.000000,60
fa1,0.961039,11
ae3,0.953125,32
pge,0.906250,24
6te,0.900000,11
...,...,...
6f9,0.608392,11
8o4,0.607639,36
fyo,0.600000,12
tde,0.600000,10


In [28]:
filtered_rec = avg_score_rec_df[(avg_score_rec_df['score'] >= 0.6) &
                                (avg_score_rec_df['num sample'] >= 10)]
filtered_rec

,score,num sample
1e3k_A,0.775073,13
4apu_B,0.740767,10
5l0k_B,0.728556,13
5hx8_A,0.714368,18
5khx_A,0.714178,20
4e5w_A,0.692320,22
4e4n_A,0.680265,25
1d0i_C,0.676899,11
3kba_A,0.674499,12
4ivc_B,0.674471,23


In [29]:
aro_match_lig = [lig[lig.rfind('rec')+9:lig.rfind('rec')+12] for lig in aro_match_l]
aro_match_lig = set(aro_match_lig)
aro_match_lig

{'0ex',
 '18n',
 '2c9',
 '2l1',
 '2l2',
 '2lk',
 '2wu',
 '3pf',
 '4ka',
 '4nc',
 '4wu',
 '4x3',
 '4x6',
 '6kn',
 '6mp',
 '6xb',
 '71a',
 '7bh',
 '98_',
 '9dg',
 'ade',
 'ale',
 'anp',
 'apz',
 'aqo',
 'b2d',
 'bpy',
 'd1n',
 'dnc',
 'dpz',
 'f31',
 'f47',
 'fmb',
 'g9h',
 'gun',
 'krw',
 'l5g',
 'lkg',
 'maq',
 'pm6',
 'pnk',
 'pq0',
 'tc7',
 'urf'}

In [37]:
aro_match_rec = set([lig[:6] for lig in aro_match_l])
aro_match_rec

{'1efz_A',
 '1enu_A',
 '1eob_A',
 '1eoc_A',
 '1k4h_A',
 '1k9s_D',
 '1kwc_B',
 '1oty_A',
 '1oum_A',
 '1ozm_A',
 '1pw7_A',
 '1q2r_A',
 '1s38_A',
 '1vdv_A',
 '1y5v_A',
 '1y5x_A',
 '1yw7_A',
 '2buq_A',
 '2bur_A',
 '2buu_A',
 '2ei0_A',
 '2qzr_A',
 '2vce_A',
 '2vch_A',
 '2wgj_A',
 '3b9j_C',
 '3c2y_A',
 '3eue_A',
 '3euf_A',
 '3ki1_A',
 '3ki3_A',
 '3nbq_B',
 '3ns1_L',
 '3nvv_C',
 '3nvw_C',
 '3ny8_A',
 '3p1c_B',
 '3p1f_A',
 '3rr4_A',
 '3sr6_C',
 '3zxz_A',
 '4deh_A',
 '4dei_A',
 '4eyf_A',
 '4ffs_A',
 '4fr6_A',
 '4gg9_A',
 '4gh1_A',
 '4gi4_A',
 '4k8a_B',
 '4knb_A',
 '4lbu_A',
 '4mrd_A',
 '4mrf_A',
 '4mrg_A',
 '4np3_A',
 '4nyw_A',
 '4ouf_B',
 '4puk_A',
 '4q4o_A',
 '4q4p_A',
 '4q4s_A',
 '4q8t_A',
 '4q8u_A',
 '4q8v_A',
 '4q8w_A',
 '4rl2_B',
 '4ts8_A',
 '4xuc_A',
 '4xud_A',
 '4xue_B',
 '4yk0_D',
 '4yur_A',
 '5bzc_A',
 '5bze_A',
 '5bzf_A',
 '5bzg_A',
 '5bzj_A',
 '5bzk_A',
 '5bzl_A',
 '5bzp_A',
 '5bzq_A',
 '5bzr_A',
 '5bzs_A',
 '5bzt_A',
 '5egr_A',
 '5eob_A',
 '5ep7_A',
 '5eyc_A',
 '5i00_A',
 '5i02_A',

In [36]:
[lig for lig in filtered_lig.index.tolist() if lig in aro_match_lig]

['4nc', '18n', 'pm6', 'pq0', '6kn', '2c9', 'dpz', 'apz', '4wu']

In [38]:
[rec for rec in filtered_rec.index.tolist() if rec in aro_match_rec]

['3ny8_A']